The code in this notebook was used to convert the *matches.json* file to a Pandas dataframe. The dataframe was saved to the *matches.pickle* file.

In [1]:
import json
import pickle

import pandas as pd

In [6]:
# Open JSON file
with open("matches.json") as jfile:
    matches = json.load(jfile)

In [7]:
# Extract metadata and convert to dataframe.
match_meta = pd.DataFrame([
    {key: val for key, val in mtch.items() 
     if not isinstance(val, list) and not isinstance(val, dict)}
    for mtch in matches])

In [8]:
# Convert time columns to datetime objects (from unix timestamps) and
#   convert to local time from UTC
for col in match_meta.columns:
    if col[-4:] == "time":
        match_meta[col] = (pd.to_datetime(match_meta[col], unit="s") -
                           pd.Timedelta(hours=8))

In [9]:
match_meta.head()

,actual_time,comp_level,event_key,key,match_number,post_result_time,predicted_time,set_number,time,winning_alliance
0,2020-03-01 15:18:36,f,2020wasno,2020wasno_f1m1,1,2020-03-01 15:24:06,2020-03-01 15:20:22,1,2020-03-01 15:06:00,red
1,2020-03-01 15:56:12,f,2020wasno,2020wasno_f1m2,2,2020-03-01 15:59:33,2020-03-01 15:57:20,1,2020-03-01 15:13:00,red
2,2020-03-01 13:03:52,qf,2020wasno,2020wasno_qf1m1,1,2020-03-01 13:07:43,2020-03-01 13:04:05,1,2020-03-01 13:00:00,red
3,2020-03-01 13:38:43,qf,2020wasno,2020wasno_qf1m2,2,2020-03-01 13:42:11,2020-03-01 13:39:07,1,2020-03-01 13:28:00,red
4,2020-03-01 13:13:35,qf,2020wasno,2020wasno_qf2m1,1,2020-03-01 13:16:28,2020-03-01 13:13:12,2,2020-03-01 13:07:00,blue


In [10]:
# Set the key column as the index
match_meta.set_index("key", inplace=True)

In [11]:
match_meta.head()

,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,set_number,time,winning_alliance
key,,,,,,,,,
2020wasno_f1m1,2020-03-01 15:18:36,f,2020wasno,1,2020-03-01 15:24:06,2020-03-01 15:20:22,1,2020-03-01 15:06:00,red
2020wasno_f1m2,2020-03-01 15:56:12,f,2020wasno,2,2020-03-01 15:59:33,2020-03-01 15:57:20,1,2020-03-01 15:13:00,red
2020wasno_qf1m1,2020-03-01 13:03:52,qf,2020wasno,1,2020-03-01 13:07:43,2020-03-01 13:04:05,1,2020-03-01 13:00:00,red
2020wasno_qf1m2,2020-03-01 13:38:43,qf,2020wasno,2,2020-03-01 13:42:11,2020-03-01 13:39:07,1,2020-03-01 13:28:00,red
2020wasno_qf2m1,2020-03-01 13:13:35,qf,2020wasno,1,2020-03-01 13:16:28,2020-03-01 13:13:12,2,2020-03-01 13:07:00,blue


In [14]:
# Extract the teams
teams = [{alliance + str(idx+1): mtch["alliances"][alliance]["team_keys"][idx]
          for alliance in ["red", "blue"] for idx in range(3)}
         for mtch in matches]
teams[:2]

[{'red1': 'frc4911',
  'red2': 'frc2910',
  'red3': 'frc4173',
  'blue1': 'frc2930',
  'blue2': 'frc2976',
  'blue3': 'frc4918'},
 {'red1': 'frc4911',
  'red2': 'frc2910',
  'red3': 'frc4173',
  'blue1': 'frc2930',
  'blue2': 'frc2976',
  'blue3': 'frc4918'}]

In [15]:
# Extract the scores
scores = [{alliance + "_score": mtch["alliances"][alliance]["score"]
           for alliance in ["red", "blue"]} for mtch in matches]
scores[:2]

[{'red_score': 148, 'blue_score': 136}, {'red_score': 163, 'blue_score': 137}]

In [16]:
# Extract the keys
keys = [{"key": mtch["key"]} for mtch in matches]
keys[:2]

[{'key': '2020wasno_f1m1'}, {'key': '2020wasno_f1m2'}]

In [19]:
# Combine teams, scores, and keys into a single dataframe.
match_teams = (
    pd.DataFrame(
        [{**keys[idx], **teams[idx], **scores[idx]}
         for idx in range(len(keys))])
    .set_index("key"))

In [20]:
match_teams.head()

,red1,red2,red3,blue1,blue2,blue3,red_score,blue_score
key,,,,,,,,
2020wasno_f1m1,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,148,136
2020wasno_f1m2,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,163,137
2020wasno_qf1m1,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,193,122
2020wasno_qf1m2,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,140,95
2020wasno_qf2m1,frc4089,frc4915,frc4309,frc1318,frc4513,frc7461,89,134


In [21]:
# Merge the two dataframes
matches = match_meta.merge(match_teams, on="key")

In [22]:
matches.head()

,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,set_number,time,winning_alliance,red1,red2,red3,blue1,blue2,blue3,red_score,blue_score
key,,,,,,,,,,,,,,,,,
2020wasno_f1m1,2020-03-01 15:18:36,f,2020wasno,1,2020-03-01 15:24:06,2020-03-01 15:20:22,1,2020-03-01 15:06:00,red,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,148,136
2020wasno_f1m2,2020-03-01 15:56:12,f,2020wasno,2,2020-03-01 15:59:33,2020-03-01 15:57:20,1,2020-03-01 15:13:00,red,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,163,137
2020wasno_qf1m1,2020-03-01 13:03:52,qf,2020wasno,1,2020-03-01 13:07:43,2020-03-01 13:04:05,1,2020-03-01 13:00:00,red,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,193,122
2020wasno_qf1m2,2020-03-01 13:38:43,qf,2020wasno,2,2020-03-01 13:42:11,2020-03-01 13:39:07,1,2020-03-01 13:28:00,red,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,140,95
2020wasno_qf2m1,2020-03-01 13:13:35,qf,2020wasno,1,2020-03-01 13:16:28,2020-03-01 13:13:12,2,2020-03-01 13:07:00,blue,frc4089,frc4915,frc4309,frc1318,frc4513,frc7461,89,134


In [23]:
# Save to file
with open("matches.pickle", "wb") as pfile:
    pickle.dump(matches, pfile)

In [24]:
# Verify file saved
with open("matches.pickle", "rb") as pfile:
    matches2 = pickle.load(pfile)
matches2.head()

,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,set_number,time,winning_alliance,red1,red2,red3,blue1,blue2,blue3,red_score,blue_score
key,,,,,,,,,,,,,,,,,
2020wasno_f1m1,2020-03-01 15:18:36,f,2020wasno,1,2020-03-01 15:24:06,2020-03-01 15:20:22,1,2020-03-01 15:06:00,red,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,148,136
2020wasno_f1m2,2020-03-01 15:56:12,f,2020wasno,2,2020-03-01 15:59:33,2020-03-01 15:57:20,1,2020-03-01 15:13:00,red,frc4911,frc2910,frc4173,frc2930,frc2976,frc4918,163,137
2020wasno_qf1m1,2020-03-01 13:03:52,qf,2020wasno,1,2020-03-01 13:07:43,2020-03-01 13:04:05,1,2020-03-01 13:00:00,red,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,193,122
2020wasno_qf1m2,2020-03-01 13:38:43,qf,2020wasno,2,2020-03-01 13:42:11,2020-03-01 13:39:07,1,2020-03-01 13:28:00,red,frc4911,frc2910,frc4173,frc4512,frc949,frc4131,140,95
2020wasno_qf2m1,2020-03-01 13:13:35,qf,2020wasno,1,2020-03-01 13:16:28,2020-03-01 13:13:12,2,2020-03-01 13:07:00,blue,frc4089,frc4915,frc4309,frc1318,frc4513,frc7461,89,134
